In [70]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [71]:
search_term1="COVID"
search_term2="stroke"
url="https://pubmed.ncbi.nlm.nih.gov/?term=%22{}%22+%22{}%22&size=200".format(search_term1,search_term2)
url2="https://pubmed.ncbi.nlm.nih.gov/"

r1 = requests.get(url)
all_papers = r1.content

soup1 = BeautifulSoup(all_papers, 'html5lib')

papers = soup1.find_all('a', class_='labs-docsum-title')

link_abstracts={}

for paper in papers:
    link=url2+paper['href']
    r2 = requests.get(link)
    paper_content = r2.content
    soup2 = BeautifulSoup(paper_content, 'html5lib')
    abstract = soup2.find_all('div', class_='abstract-content')
    if not bool(abstract):
        continue
    link_abstracts[link]=abstract[0]

In [72]:
# from https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    re.sub(clean, '', text)
    return text.replace('\n', "")

In [75]:
filename="COVID+stroke keywords"
file_ext=filename+".csv"
for key in link_abstracts:
    raw_text = str(link_abstracts[key])
    abstract_text = remove_html_tags(raw_text)
    link_abstracts[key] = abstract_text

with open(file_ext, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key, value in link_abstracts.items():
           writer.writerow([key, value])